In [3]:
import os, math, glob, json, random
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Set random seeds
tf.random.set_seed(666)
np.random.seed(666)

## Data Summary

#### Import Data

In [7]:
train_images_path = './train/'
test_images_path  = './test/'
train_labels_csv  = './train_labels.csv'

IMG_SIZE = 96
CHANNELS = 3
AUTOTUNE = tf.data.AUTOTUNE

['./test/5b00f00416e1a528723aa55437144ac5ab0aa2f0.tif',
 './test/b5a489eeec3b527376545944a5752d5c6cf42b67.tif',
 './test/215bdc58a5dcf7974aaf6f3b35256a24ed045d83.tif',
 './test/940650a72d99e51d98920a78f2746aced50e2b23.tif',
 './test/85cbdcb02982ef12f2ed665ffd90990c37f6af1d.tif',
 './test/fc86640eff420ed9ea8201d641c23dd6281bef6c.tif',
 './test/bcf3f1cfa028d031dfc6715c8201e73f038d0cd7.tif',
 './test/2e7647030e949541de4e06bb7919ac2349eca19b.tif',
 './test/614a1c7a60021a93700e1c42fbb56a88bc25d47c.tif',
 './test/574b30e69d14be04685eae0c4d9c5babfa2ffeed.tif',
 './test/e960adba79ca2c8c9a192160beca120a33523f6d.tif',
 './test/5acec5d1d0b3d05d88556ba85a985ed109aa2387.tif',
 './test/1c53bfcab1db5a0ebef079b2c50dd335dbfd704d.tif',
 './test/ceeb441190f77da8717c7f44ddfe0fb5fc805dd5.tif',
 './test/3c02240799a08200c5e37fdc289236186e726b2c.tif',
 './test/43f16ab2632963ec3195a8e3783bc66055cdd6de.tif',
 './test/b2b0531c6052da922cde386ef9f976620a84fe18.tif',
 './test/158c432e891d09f36758a38344f138a79bf3e90

In [19]:
# Load training labels
labels_df = pd.read_csv(train_labels_csv)

# Create a list of the file paths to the images in the train and test folders
train_file_paths = sorted(glob.glob(os.path.join(train_images_path, '*.tif')))
test_file_paths = sorted(glob.glob(os.path.join(test_images_path, '*.tif')))

In [21]:
test_file_paths

['./test/00006537328c33e284c973d7b39d340809f7271b.tif',
 './test/0000ec92553fda4ce39889f9226ace43cae3364e.tif',
 './test/00024a6dee61f12f7856b0fc6be20bc7a48ba3d2.tif',
 './test/000253dfaa0be9d0d100283b22284ab2f6b643f6.tif',
 './test/000270442cc15af719583a8172c87cd2bd9c7746.tif',
 './test/000309e669fa3b18fb0ed6a253a2850cce751a95.tif',
 './test/000360e0d8358db520b5c7564ac70c5706a0beb0.tif',
 './test/00040095a4a671280aeb66cb0c9231e6216633b5.tif',
 './test/000698b7df308d75ec9559ef473a588c513a68aa.tif',
 './test/0006e1af5670323331d09880924381d67d79eda0.tif',
 './test/000997a6038fa7441aa0111ac456255060a354c4.tif',
 './test/000c8db3e09f1c0f3652117cf84d78aae100e5a7.tif',
 './test/000de14191f3bab4d2d6a7384ca0e5aa5dc0dffe.tif',
 './test/000e6341cf18365d35b40f4991002fec8834afc0.tif',
 './test/0010e2887e0b977fcdfdf4c50564fafbbc2b6208.tif',
 './test/0010e7eaa3d8e14203cd3900b739d8bf0f0b67f0.tif',
 './test/001161a2eca200f565f12870048a78fa5b320dee.tif',
 './test/0011807dd1e3306ff3f7a755fd3efbefa2901dc

In [ ]:
# Add the filename paths to labels_df
labels_df['filepath'] = labels_df['id'].apply(lambda x: os.path.join(train_images_path, f'{x}.tif'))

# Ensure the created filename paths exist in the train folder
labels_df['filepath'].apply(os.path.exists)

0         True
1         True
2         True
3         True
4         True
          ... 
220020    True
220021    True
220022    True
220023    True
220024    True
Name: filepath, Length: 220025, dtype: bool

In [ ]:

# Append t
labels_df[labels_df['filepath'].apply(os.path.exists)].reset_index(drop=True)

In [12]:
labels_df['filepath'].apply(os.path.exists)

0         True
1         True
2         True
3         True
4         True
          ... 
220020    True
220021    True
220022    True
220023    True
220024    True
Name: filepath, Length: 220025, dtype: bool

In [ ]:

labels_df = labels_df[labels_df['filepath'].apply(os.path.exists)].reset_index(drop=True)

train_files = labels_df['filepath'].tolist()
train_labels_vec = labels_df['label'].astype(np.int8).values

test_files = sorted(glob.glob(os.path.join(test_images_path, '*.tif')))

print(f'train images: {len(train_files)}  test images: {len(test_files)}')

## Results & Analysis
- Discuss and interpret results

#### Kaggle Submission

In [ ]:
# Get best model
# best_model = 

# Predict labels for test images


# Convert prediction into Kaggle submission format


In [ ]:
# Image of Kaggle submission


## Conclusion
- Summarize project
- Learnings and takeaways
- What did and did not help improve the performance of your models
- What improvements could you try in the future